In [40]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import sklearn
import time

In [30]:
amz = pd.read_json('Musical_Instruments_5.json', lines=True)

amz = amz[['reviewText','overall']]
amz.columns = ['text','rating']
amz['rating'] = np.where(amz['rating'] < 3, 0 ,1)

In [31]:
pd.options.display.max_colwidth = 200
display(amz)

,text,rating
0,"Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so ...",1
1,"The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even better than I had expected.As an added bonus, one of the sc...",1
2,"The primary job of this device is to block the breath that would otherwise produce a popping sound, while allowing your voice to pass through with no noticeable reduction of volume or high frequen...",1
3,Nice windscreen protects my MXL mic and prevents pops. Only thing is that the gooseneck is only marginally able to hold the screen in position and requires careful positioning of the clamp to avoi...,1
4,This pop filter is great. It looks and performs like a studio filter. If you're recording vocals this will eliminate the pops that gets recorded when you sing.,1
5,So good that I bought another one. Love the heavy cord and gold connectors. Bass sounds great. I just learned last night how to coil them up. I guess I should read instructions more carefully....,1
6,"I have used monster cables for years, and with good reason. The lifetime warranty is worth the price alone. Simple fact: cables break, but getting to replace them at no cost is where it's at.",1
7,I now use this cable to run from the output of my pedal chain to the input of my Fender Amp. After I bought Monster Cable to hook up my pedal board I thought I would try another one and update my ...,1
8,Perfect for my Epiphone Sheraton II. Monster cables are well constructed. I have several and never had any problems with any of them over the years. Got this one because I wanted the 90 degree ...,1
9,Monster makes the best cables and a lifetime warranty doesnt hurt either. This isnt their top of the line series but it works great with my bass guitar rig and has for some time. You cant go wrong...,1


In [36]:
keywords = ['great', 'exactly', 'promise', 'buy',
            'price', 'free', 'quality', 'again', 
            'true', 'brand', 'love', 'well', 
            'durable', 'last']

for key in keywords:
    # Note that we add spaces around the key so that we're getting the word, not just pattern matching.
    amz[str(key)] = amz.text.str.contains(' ' + str(key) + ' ', case=False)

X = amz[keywords]
y = amz['rating']

display(amz.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 16 columns):
text       10261 non-null object
rating     10261 non-null int32
great      10261 non-null bool
exactly    10261 non-null bool
promise    10261 non-null bool
buy        10261 non-null bool
price      10261 non-null bool
free       10261 non-null bool
quality    10261 non-null bool
again      10261 non-null bool
true       10261 non-null bool
brand      10261 non-null bool
love       10261 non-null bool
well       10261 non-null bool
durable    10261 non-null bool
last       10261 non-null bool
dtypes: bool(14), int32(1), object(1)
memory usage: 260.6+ KB


None

In [37]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(X, y)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

total = data.shape[0]
wrong = (target != y_pred).sum()
right = (target == y_pred).sum()
# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    total,
    wrong
))
print('Accuracy: ', right/total )

Number of mislabeled points out of a total 10261 points : 467
Accuracy:  0.9544878666796609


In [94]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X_train , X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

start_time = time.time()

# We'll make 5000 iterations, use  10-deep trees, set our loss function, and max leaf-nodes.
params = {'n_estimators': 1500,
          'max_depth': 8,
          'subsample' : 1,
          'loss': 'exponential', #exponential,deviance
          'max_leaf_nodes': 8}

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(X_train, y_train)

print("--- %s seconds for model fit ---" % (time.time() - start_time))

predict_train = clf.predict(X_train)
predict_test = clf.predict(X_test)

# Accuracy tables.
table_train = pd.crosstab(y_train, predict_train, margins=True)
table_test = pd.crosstab(y_test, predict_test, margins=True)
train_tI_errors = table_train.loc[0,1] / table_train.loc['All','All']
train_tII_errors = table_train.loc[1,0] / table_train.loc['All','All']
test_tI_errors = table_test.loc[0,1]/table_test.loc['All','All']
#test_tII_errors = table_test.loc[1,0]/table_test.loc['All','All']

print((
    'Training set accuracy:\n'
    '  Percent Type I errors: {}\n'
    '  Percent Type II errors: {}\n\n'
    'Test set accuracy:\n'
    '  Percent Type I errors: {}\n'
    '  Percent Type II errors: {}\n'
).format(train_tI_errors, train_tII_errors, test_tI_errors, 0.0))

print('Training Accuracy:\n   ', clf.score(X_train, y_train))
print('Testing Accuracy:\n   ', clf.score(X_test, y_test))


#start_time = time.time()
#cvs = cross_val_score(clf,X, Y, cv=8)
#print('\nCross Validation Score:\n    {} +/- {}'.format(round(cvs.mean(),6), round(cvs.std(),6)))
#print("--- %s seconds for cross-val score ---" % (time.time() - start_time))

--- 9.940999746322632 seconds for model fit ---
Training set accuracy:
  Percent Type I errors: 0.04446881091617934
  Percent Type II errors: 0.0

Test set accuracy:
  Percent Type I errors: 0.04578665367754506
  Percent Type II errors: 0.0

Training Accuracy:
    0.9555311890838206
Testing Accuracy:
    0.9542133463224549


In [97]:
from sklearn.model_selection import cross_val_score

start_time = time.time()
cvs = cross_val_score(clf,X, y, cv=8)
print('\nCross Validation Score:\n    {} +/- {}'.format(round(cvs.mean(),6), round(cvs.std(),6)))
print("--- %s seconds for cross-val score ---" % (time.time() - start_time))


Cross Validation Score:
    0.953319 +/- 0.001238
--- 99.51279997825623 seconds for cross-val score ---


In [85]:
print(table_train)
print(table_train.loc[0,1])
print(table_train.loc[1,0])

print(table_test)
print(table_test.loc[0,1])
print(table_test.loc[1,0])

col_0    0     1   All
rating                
0       10   229   239
1        0  4891  4891
All     10  5120  5130
229
0
col_0    0     1   All
rating                
0        1   227   228
1       16  4887  4903
All     17  5114  5131
227
16
